<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg"  width=400></p>

<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import torch

### 1. Нахождение сложной производной

Найдите производную по x от функции 
$$\sin\left(\tan(x)\frac{x^2}{y} + \ln(e^{-x^2 + 3}+x^3y)\right)\tan(x^2e^{x^9})$$

При этом надо пользоваться встроенным в PyTorch autograd. Численное вычисление производной может не дать нужный результат.

In [2]:
def find_x_derivative(x, y):
    x = torch.autograd.Variable(torch.Tensor([x]),requires_grad = True)
    out = torch.sin( torch.tan(x)*(x * x / y) + torch.log(torch.exp(-x * x + 3) + x * x * x * y) ) * torch.tan( x * x * (torch.exp(x**9)))
    out.backward()
    #print(x)
    return x.grad

In [3]:
print(find_x_derivative(1, 21))

tensor([-9.0207])


# 2. Нахождение косинусной близости

Вам даны две матрицы A и B. Необходимо посчитать косинусную близость между строчками матрицы A и столбцами матрицы B. Ответ - матрица чисел, где номер строки - номер строки из матрицы А, а номер столбца - номер столбца из В, от которых бралась косинусная близость.

Напомним, что косинусная близость двух векторов - косинус угла между ними. В n-мерном пространстве косинус угла между веткорами удобнее всего через скалярное произведение:
$$\cos(angle(x, y)) = \frac{x \cdot y}{\left\|x\right\| \left\|y\right\|}$$

(Наша операция очень похожа на умножение матриц)

In [4]:
def get_cos_sim(A, B):
    """
        A, B - torch float tensors
    """
    res = torch.mm(a / torch.norm(a, dim = 1, keepdim = True), b / torch.norm(b, dim = 0, keepdim = True))
    
    return res

In [5]:
a = torch.FloatTensor([[1, -47, 25, -3], [10, 17, -15, 22], [-3, -7, 26, 36], [12, -27, -42, 0]])
print(a)
b = torch.FloatTensor([[-50, -13, 1, 10, 1242], [21, 48, -13, -14, -20], [20, 15, 11, 43, 11], [11, 103, 147, 27, -8]])
print(b)
print(torch.mean(get_cos_sim(a, b)))

tensor([[  1., -47.,  25.,  -3.],
        [ 10.,  17., -15.,  22.],
        [ -3.,  -7.,  26.,  36.],
        [ 12., -27., -42.,   0.]])
tensor([[-5.0000e+01, -1.3000e+01,  1.0000e+00,  1.0000e+01,  1.2420e+03],
        [ 2.1000e+01,  4.8000e+01, -1.3000e+01, -1.4000e+01, -2.0000e+01],
        [ 2.0000e+01,  1.5000e+01,  1.1000e+01,  4.3000e+01,  1.1000e+01],
        [ 1.1000e+01,  1.0300e+02,  1.4700e+02,  2.7000e+01, -8.0000e+00]])
tensor(0.1498)


# 3. Линейная регрессия

Раньше мы самостоятельно считали производные, чтобы находить веса линейной регрессии с помощью градиентного спуска. Теперь нам нужно использовать для этого PyTorch и его autograd. 

**Важно**: на самом деле .backward не обновляет содержимое матриц с производными (some_tensor.grad), а прибавляет к ним только что посчитаные значения проивзодных. Это значит, что вызвав .backward дважды, вы получите удвоенную производную. Так как мы обновляем веса в цикле и много раз вызываем .backward, то очень быстро мы получим мусор в some_tensor.grad, если не будем его каждый раз обнулять. Таким образом, в конц итериации после использования производных обнулите значения в матрице производных для всех нужных Вам переменных. Делается это вот так 
> some\_tensor.grad.data.zero_()

In [18]:
class LinearRegression:
    def get_loss(self, preds, y):
        """
            @param preds: предсказания модели
            @param y: истиные значения
            @return mse: значение MSE на переданных данных
        """
        # возьмите средний квадрат ошибки по всем выходным переменным
        # то есть сумму квадратов ошибки надо поделить на количество_элементов * количество_таргетов
        loss = preds - y
        return torch.sum(loss * loss) / loss.numel()
    
    def init_weights(self, input_size, output_size):
        """
            Инициализирует параметры модели
            W - матрица размерности (input_size, output_size)
            инициализируется рандомными числами из
            uniform распределения (torch.rand())
            b - вектор размерности (1, output_size)
            инициализируется нулями
        """
        torch.manual_seed(0) #необходимо для воспроизводимости результатов
        self.W = torch.rand(input_size, output_size,  requires_grad = True)
        self.b = torch.zeros(1, output_size, requires_grad = True)

    def fit(self, X, y, num_epochs = 1000, lr = 0.001):
        """
            Обучение модели линейной регрессии методом градиентного спуска
            @param X: размерности (num_samples, input_shape)
            @param y: размерности (num_samples, output_shape)
            @param num_epochs: количество итераций градиентного спуска
            @param lr: шаг градиентного спуска
            @return metrics: вектор значений MSE на каждом шаге градиентного
            спуска.
        """
        self.init_weights(X.shape[1], y.shape[1])
        metrics = []
        for _ in range(num_epochs):
            preds = self.predict(X)
            # сделайте вычисления градиентов c помощью Pytorch и обновите веса
            # осторожнее, оберните вычитание градиента в 
#                 with torch.no_grad():
#                     #some code
            # иначе во время прибавления градиента к переменной создастся очень много нод в дереве операций
            # и ваши модели в будущем будут падать от нехватки памяти
            self.get_loss(preds, y).backward()
            with torch.no_grad():
              self.W -= self.W.grad * lr
              self.b -= self.b.grad * lr
              self.W.grad.zero_()
              self.b.grad.zero_()
            metrics.append(self.get_loss(preds, y).data)
        return metrics

    def predict(self, X):
        return X @ self.W + self.b

1. Сгенерируйте данные с помощью make_regression с параметрами n_targets=3, n_features=2, noise=10, random_state=42. 
2. Обучите модель линейной регрессии, оставив в fit параметры num_epochs и lr по умолчанию (обратите внимание, что перед обучением нужно привести данные к типу, использующимся в torch) 
2. Посчитайте среднее значение метрики MSE по всем итерациям цикла в fit (массив из значений MSE на каждой итерации возвращается из метода fit). Ответом, который необходимо сдать в систему, будет число, округленное до 3х знаков после запятой.

In [19]:
X, Y = datasets.make_regression(n_targets = 3, n_features = 2, noise = 10, random_state = 42)
X = torch.from_numpy(X).float()
Y = torch.from_numpy(Y).float()
model = LinearRegression()
mse = model.fit(X, Y)

In [20]:
print(np.mean(mse))

4256.561


Здесь предлагаем протестировать метод predict удобным вам образом.

In [21]:
print(model.predict(X))

tensor([[  6.8541,   0.5543,  -1.5183],
        [-56.6351, -42.0323, -39.0554],
        [-62.4823, -31.9040, -25.9731],
        [-42.1749, -32.7359, -30.9805],
        [  8.4156,  15.5514,  15.8258],
        [-53.5753, -26.0870, -20.8924],
        [ 45.9922,  33.3924,  29.3737],
        [ 31.4741,  14.5750,  10.1026],
        [ 66.9552,  45.2404,  39.1624],
        [-16.9052, -11.5615, -11.0675],
        [-11.2921,  -0.6256,   0.6923],
        [ 43.6821,  15.3659,   8.6121],
        [ 36.0577,  18.8220,  14.1928],
        [  3.1521,  -2.0366,  -3.8338],
        [-83.2639, -57.4035, -51.8676],
        [ 18.9995,  11.4185,   8.8614],
        [-39.4610, -19.9890, -16.5138],
        [-21.7515, -24.2949, -25.0953],
        [ 44.3900,  25.7567,  20.7032],
        [ -3.4867, -10.4673, -12.4412],
        [ 30.3031,  30.5916,  29.1889],
        [-17.6429,  -7.5654,  -6.2171],
        [-12.7887,  -4.2539,  -3.2819],
        [ 29.5546,  22.2569,  19.5261],
        [ -7.4601,   3.1827,   4.4151],
